In [1]:
import sys
import matplotlib.pyplot as plt
import h5py
from tqdm import tqdm
import librosa
import numpy as np
from keras.utils.np_utils import to_categorical
from sklearn.utils import shuffle
import cv2
import torch

classes = {
    "Rhinolophus ferrumequinum": 0,
    "Rhinolophus hipposideros": 1,
    "Myotis daubentonii": 2,
    "Myotis brandtii": 3,
    "Myotis mystacinus": 4,
    "Myotis emarginatus": 5,
    "Myotis nattereri": 6,
    #"Myotis bechsteinii": 7,
    "Myotis myotis": 7,
    "Myotis dasycneme": 8,
    "Nyctalus noctula": 9,
    "Nyctalus leisleri": 10,
    "Pipistrellus pipistrellus": 11,
    "Pipistrellus nathusii": 12,
    "Pipistrellus kuhlii": 13,
    "Eptesicus serotinus": 14,
    "Eptesicus nilssonii": 15,
    #"Plecotus auritus": 16,
    #"Plecotus austriacus": 16,
    #"Barbastella barbastellus": 16,
    #"Tadarida teniotis": 16,
    "Miniopterus schreibersii": 16,
    #"Hypsugo savii": 18,
    "Vespertilio murinus": 17,
}

2022-11-10 11:38:50.765316: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-10 11:38:51.220486: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-10 11:38:54.088332: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ffundel/.local/lib:
2022-11-10 11:38:54.089152: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cann

In [2]:
skiba_path = "prepared_signal.h5"
skiba_hf = h5py.File(skiba_path, 'a')
skiba_train = skiba_hf.require_group("train")
skiba_test = skiba_hf.require_group("test")
skiba_val = skiba_hf.require_group("val")

In [3]:
chirovox_path = "prepared_signal_chirovox.h5"
chirovox_hf = h5py.File(chirovox_path, 'r')
chirovox_test = chirovox_hf.require_group("test")

In [9]:
def mergeDS(ds1, ds2, group):
    for species in tqdm(list(classes)):
        s1 = ds1.get(species)
        s2 = ds2.get(species)
        group.create_dataset(species, data=np.concatenate([s1, s2]))
        
def copyDS(ds, group):
    for species in tqdm(list(classes)):
        s = ds.get(species)
        if s:
            group.create_dataset(species, data=s)
        else:
            group.create_dataset(species, data=[])

In [10]:
hf = h5py.File("skiba_chirovox2", 'w')
train_set = hf.require_group("train")
test_set = hf.require_group("test")
val_set = hf.require_group("val")

In [12]:
#mergeDS(skiba_train, skiba_test, train_set)
copyDS(chirovox_test, test_set)
copyDS(skiba_val, val_set)

 17%|███████▎                                    | 3/18 [00:00<00:02,  6.41it/s]

Eptesicus nilssonii


100%|███████████████████████████████████████████| 18/18 [00:04<00:00,  4.44it/s]


In [7]:
hf.close()